# More JOIN operations

![rel](https://sqlzoo.net/w/images/5/50/Movie2-er.png)

## 1. 1962 movies

List the films where the **yr** is 1962 [Show **id, title**]

## 2. When was Citizen Kane released?

Give year of 'Citizen Kane'.

## 3. Star Trek movies

List all of the Star Trek movies, include the **id**, **title** and **yr** (all of these movies include the words Star Trek in the title). Order results by year.

## 4. id for actor Glenn Close

What **id** number does the actor 'Glenn Close' have?

## 5. id for Casablanca

What is the **id** of the film 'Casablanca'

## 6. Cast list for Casablanca

Obtain the cast list for 'Casablanca'.

> _what is a cast list?_  
> The cast list is the names of the actors who were in the movie.

Use **movieid=11768**, (or whatever value you got from the previous question)

## 7. Alien cast list

Obtain the cast list for the film 'Alien'

## 8. Harrison Ford movies

List the films in which 'Harrison Ford' has appeared

## 9. Harrison Ford as a supporting actor

List the films where 'Harrison Ford' has appeared - but not in the starring role. [Note: the ord field of casting gives the position of the actor. If ord=1 then this actor is in the starring role]

## 10. Lead actors in 1962 movies

List the films together with the leading star for all 1962 films.

## 11. Busy years for Rock Hudson

Which were the busiest years for 'Rock Hudson', show the year and the number of movies he made each year for any year in which he made more than 2 movies.

## 12. Lead actor in Julie Andrews movies

List the film title and the leading actor for all of the films 'Julie Andrews' played in.

> _Did you get "Little Miss Marker twice"?_   
> Julie Andrews starred in the 1980 remake of Little Miss Marker and not the original(1934).
>
> Title is not a unique field, create a table of IDs in your subquery

## 13. Actors with 15 leading roles

Obtain a list, in alphabetical order, of actors who've had at least 15 **starring** roles.

## 14.
List the films released in the year 1978 ordered by the number of actors in the cast, then by title.

## 15.

List all the people who have worked with 'Art Garfunkel'.